In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold, GroupKFold, StratifiedKFold
import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import seaborn as sns
from tqdm import tqdm
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.preprocessing import LabelEncoder
DATA_PATH = "../input/ashrae-energy-prediction/"
LEAKDATA_PATH = "../input/leakdata/"



In [19]:
from datetime import datetime

In [2]:
#train = pd.read_pickle("train.pkl")
#test = pd.read_pickle("test.pkl")

In [3]:
train = pd.read_csv(DATA_PATH + "train.csv")
#test = pd.read_csv(DATA_PATH + "test.csv")
building_df = pd.read_csv(DATA_PATH + "building_metadata.csv")
weather_train = pd.read_csv(DATA_PATH + "weather_train.csv")
weather_test = pd.read_csv(DATA_PATH + "weather_test.csv")
train = train.merge(building_df, left_on = "building_id", right_on = "building_id", how = "left")
#test = test.merge(building_df, left_on = "building_id", right_on = "building_id", how = "left")
train = train.merge(weather_train, left_on = ["site_id", "timestamp"], right_on = ["site_id", "timestamp"], how='left')
#test = test.merge(weather_test, left_on = ["site_id", "timestamp"], right_on = ["site_id", "timestamp"], how='left')

In [4]:
for df in [train]:
#for df in [train, test]:
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    df["hour"] = df["timestamp"].dt.hour
    df["day"] = df["timestamp"].dt.day
    df["weekend"] = df["timestamp"].dt.weekday
    df["month"] = df["timestamp"].dt.month
    df["year"] = df["timestamp"].dt.year
    df["yearmonth"] = (2018-df["timestamp"].dt.year) * 12 + df["timestamp"].dt.month

In [5]:
le = LabelEncoder()
train["primary_use"] = le.fit_transform(train["primary_use"])
#test["primary_use"] = le.fit_transform(test["primary_use"])

In [6]:
func = {'air_temperature':['mean', 'median', 'min', 'max'],
        'cloud_coverage':['mean', 'median', 'min', 'max'],
        'dew_temperature':['mean', 'median', 'min', 'max']}
g1 = train.groupby(['building_id', 'year', 'month', 'day']).agg(func)
#g2 = test.groupby(['building_id', 'year', 'month', 'day']).agg(func)
g1.columns = ['_'.join(col) for col in g1.columns]
#g2.columns = ['_'.join(col) for col in g2.columns]
train = train.merge(g1, on=['building_id', 'year', 'month', 'day'], how='left')
#test = test.merge(g2, on=['building_id', 'year', 'month', 'day'], how='left')

In [7]:
enc = pd.concat([train[['building_id']], test[['building_id']]])['building_id'].value_counts().to_dict()
train['building_id_count'] = train['building_id'].map(enc)
#test['building_id_count'] = test['building_id'].map(enc)

NameError: name 'test' is not defined

In [ ]:
g = train.groupby(['building_id', 'meter']).agg({'meter_reading':['var', 'std', 'min', 'max']})
g.columns = ['_'.join(col) for col in g.columns]
train = train.merge(g, on=['building_id', 'meter'], how='left')
#test = test.merge(g, on=['building_id', 'meter'], how='left')

In [ ]:
g = train.groupby(['building_id', 'meter', 'month']).agg({'meter_reading':['std']})
g.columns = ['_'.join(col) for col in g.columns]
g = g.groupby(['building_id', 'meter']).agg({'meter_reading_std':['std']})
g.columns = ['_'.join(col) for col in g.columns]
train = train.merge(g, on=['building_id', 'meter'], how='left')
#test = test.merge(g, on=['building_id', 'meter'], how='left')

In [ ]:
site4 = pd.read_csv(LEAKDATA_PATH+"site4_leak_rrr_v1.csv")
site4['timestamp'] = pd.to_datetime(site4["timestamp"])
train = train.merge(site4, on=['building_id', 'timestamp'], how='left')
#test = test.merge(site4, on=['building_id', 'timestamp'], how='left')

In [13]:
remove_features = ['year','timestamp','meter_reading', "precip_depth_1_hr", "sea_level_pressure", "wind_direction", "wind_speed"]
features = []
for col in train.columns:
    if col not in remove_features:
        features.append(col)
#for col in ['building_id', 'site_id','meter','primary_use','square_feet','year_built','floor_count']:
#    try:features.remove(col)
#    except:None

In [ ]:
train.columns

In [ ]:
train.head(1000).to_csv('../input/train_1k.csv')

In [ ]:
print(len(features))
features

In [ ]:
train[['yearmonth']].head()

In [ ]:
building_df

In [ ]:
t1 = train[train.building_id == 109]
t1.meter.unique()
t1.describe()

In [ ]:
t1[['building_id', 'meter', 'meter_reading']].groupby(['building_id', 'meter']).max()

In [ ]:
categorical_feats = []

## Preprocessing

## Delete first rows with 0

In [ ]:
train = train.query('not (building_id <= 104 & meter == 0 & timestamp <= "2016-05-21")')
train = train.query('not (building_id == 29 & meter == 0 & timestamp <= "2016-08-12")')
train = train.query('not (building_id == 29 & meter == 0 & timestamp <= "2016-08-12")')
train = train.query('not (building_id == 7 & meter == 1 & timestamp <= "2016-11-29" & timestamp >= "2016-11-03")')
train = train.query('not (building_id == 954 & timestamp <= "2016-08-09")')
train = train.query('not (building_id == 783 & meter == 2 & timestamp <= "2016-12-10")')
train = train.query('not (building_id == 1250 & meter == 2 & timestamp <= "2016-12-22")')
train = train.query('not (building_id == 1250 & meter == 2 & timestamp <= "2016-12-22")')
train = train.query('not (building_id == 1250 & meter == 0 & meter_reading == 0)')
train = train.query('not (timestamp >= "2016-09-27 23:00:00" & timestamp <= "2016-10-20 23:00:00" & building_id <= 1280 & building_id >= 1275 & meter == 0 )')
train = train.query('not (timestamp >= "2016-09-27 23:00:00" & timestamp <= "2016-10-20 23:00:00" & building_id <= 1300 & building_id >= 1294 & meter == 0 )')
train = train.query('not (building_id == 1130 & meter == 1 & timestamp < "2016-10-01")') #start
train = train.query('not (building_id == 1303 & meter == 2 & timestamp < "2016-06-07")') #start
train = train.query('not (building_id == 1303 & meter == 0 & meter_reading < 400)') #delete nulls
train = train.query('not (building_id == 1238 & meter == 0 & meter_reading < 1)') #delete nulls
train = train.query('not (building_id == 1238 & meter == 2 & meter_reading < 1)') #delete nulls
train = train.query('not (building_id == 1427 & meter == 2 & meter_reading < 1)') #delete nulls
train = train.query('not (building_id == 1425 & meter == 2 & meter_reading < 1)') #delete nulls
train = train.query('not (building_id == 1392 & meter == 2 & meter_reading < 100)') #delete nulls
train = train.query('not (building_id == 1391 & meter == 2 & meter_reading < 10)') #delete nulls
train = train.query('not (building_id == 1383 & meter == 2 & meter_reading < 10)') #delete nulls
train = train.query('not (building_id == 1383 & meter == 0 & meter_reading < 10)') #delete nulls
train = train.query('not (building_id == 1382 & meter == 2 & meter_reading < 10)') #delete nulls
train = train.query('not (building_id == 1323 & meter == 0 & meter_reading < 1)') #delete nulls
train = train.query('not (building_id == 1322 & meter == 0 & meter_reading < 1)') #delete nulls
train = train.query('not (building_id == 1321 & meter == 0 & meter_reading < 1)') #delete nulls
train = train.query('not (building_id == 1320 & meter == 0 & meter_reading < 1)') #delete nulls
train = train.query('not (building_id == 1317 & meter == 0 & meter_reading < 1)') #delete nulls
train = train.query('not (building_id == 1301 & meter == 0 & meter_reading < 1)') #delete nulls
train = train.query('not (building_id == 1288 & meter == 0 & meter_reading < 1)') #delete nulls
train = train.query('not (building_id == 1287 & meter == 2 & meter_reading < 1)') #delete nulls
train = train.query('not (building_id == 1285 & meter == 0 & meter_reading < 1)') #delete nulls
train = train.query('not (building_id == 1284 & meter == 2 & meter_reading < 1)') #delete nulls
train = train.query('not (building_id == 1233 & meter == 0 & meter_reading < 1)') #delete nulls
train = train.query('not (building_id == 1234 & meter == 0 & meter_reading < 1)') #delete nulls
train = train.query('not (building_id == 1246 & meter == 0 & meter_reading < 1)')
train = train.query('not (building_id == 1246 & meter == 1 & timestamp < "2016-03-07")')
train = train.query('not (building_id == 1246 & meter == 3 & timestamp < "2016-03-07")')
train = train.query('not (building_id == 1319 & meter == 0 & meter_reading < 1)')
train = train.query('not (building_id == 1302  & timestamp > "2016-09-28" & timestamp < "2016-10-21")')
train = train.query('not (timestamp >= "2016-09-27 23:00:00" & timestamp <= "2016-10-20 23:00:00" & building_id == 1300 & meter == 1 )')
train = train.query('not (timestamp >= "2016-09-27 23:00:00" & timestamp <= "2016-10-20 23:00:00" & building_id == 1300 & meter == 2 )')
train = train.query('not (timestamp >= "2016-09-27 23:00:00" & timestamp <= "2016-10-20 23:00:00" & building_id == 1299 & meter == 2 )')
train = train.query('not (timestamp >= "2016-09-27 23:00:00" & timestamp <= "2016-10-20 23:00:00" & building_id == 1298 & meter == 2 )')
train = train.query('not (timestamp >= "2016-09-27 23:00:00" & timestamp <= "2016-10-20 23:00:00" & building_id == 1297)')
train = train.query('not (timestamp >= "2016-09-27 23:00:00" & timestamp <= "2016-10-20 23:00:00" & building_id == 1296)')
train = train.query('not (timestamp >= "2016-09-27 23:00:00" & timestamp <= "2016-10-20 23:00:00" & building_id == 1295)')
train = train.query('not (timestamp >= "2016-09-27 23:00:00" & timestamp <= "2016-10-20 23:00:00" & building_id == 1294)')
train = train.query('not (timestamp >= "2016-09-27 23:00:00" & timestamp <= "2016-10-20 23:00:00" & building_id == 1293)')
train = train.query('not (timestamp >= "2016-09-27 23:00:00" & timestamp <= "2016-10-20 23:00:00" & building_id == 1292)')
train = train.query('not (timestamp >= "2016-09-27 23:00:00" & timestamp <= "2016-10-20 23:00:00" & building_id == 1291)')
train = train.query('not (timestamp >= "2016-07-06 23:00:00" & timestamp <= "2016-08-12 23:00:00" & building_id == 1288 & meter == 1)')
train = train.query('not (timestamp >= "2016-09-27 23:00:00" & timestamp <= "2016-10-20 23:00:00" & building_id == 1276 & meter == 3)')
train = train.query('not (timestamp >= "2016-09-27 23:00:00" & timestamp <= "2016-10-20 23:00:00" & building_id == 1275 & meter == 3)')
train = train.query('not (timestamp >= "2016-09-27 23:00:00" & timestamp <= "2016-10-20 23:00:00" & building_id == 1272)')
train = train.query('not (timestamp >= "2016-08-21 23:00:00" & timestamp <= "2016-10-20 23:00:00" & building_id == 1264)')
train = train.query('not (timestamp >= "2016-11-09 23:00:00" & timestamp <= "2016-11-21 23:00:00" & building_id == 1263 & meter == 1)')
train = train.query('not (timestamp >= "2016-11-09 23:00:00" & timestamp <= "2016-11-21 23:00:00" & building_id == 1263 & meter == 2)')
train = train.query('not (timestamp >= "2016-08-20 23:00:00" & timestamp <= "2016-09-05 23:00:00" & building_id == 1252 & meter == 3)')
train = train.query('not (timestamp >= "2016-09-27 23:00:00" & timestamp <= "2016-10-20 23:00:00" & building_id == 1223)')
train = train.query('not (timestamp >= "2016-09-27 23:00:00" & timestamp <= "2016-10-20 23:00:00" & building_id == 1225)')
train = train.query('not (timestamp >= "2016-09-27 23:00:00" & timestamp <= "2016-10-20 23:00:00" & building_id == 1226)')
train = train.query('not (timestamp >= "2016-06-22 23:00:00" & timestamp <= "2016-08-31 23:00:00" & building_id == 1232 & (meter == 1 or meter == 3))')
train = train.query('not (timestamp >= "2016-09-27 23:00:00" & timestamp <= "2016-10-20 23:00:00" & building_id == 1233 )')
train = train.query('not (timestamp >= "2016-09-27 23:00:00" & timestamp <= "2016-10-20 23:00:00" & building_id == 1234 )')
train = train.query('not (meter == 3 & timestamp >= "2016-04-03 23:00:00" & timestamp <= "2016-05-09 23:00:00" & building_id == 1235 )')
train = train.query('not (timestamp >= "2016-04-20 23:00:00" & timestamp <= "2016-05-02 23:00:00" & building_id == 1240 )')
train = train.query('not (meter == 3 & timestamp >= "2016-04-03 23:00:00" & timestamp <= "2016-05-06 23:00:00" & building_id == 1244 )')
train = train.query('not (meter == 2 & timestamp >= "2016-10-18 23:00:00" & timestamp <= "2016-11-08 23:00:00" & building_id == 1243 )')
train = train.query('not (meter == 0 & timestamp >= "2016-07-13 00:00:00" & timestamp <= "2016-11-20 00:00:00" & building_id == 1241 )')
train = train.query('not (meter == 0 & timestamp >= "2016-09-14 00:00:00" & timestamp <= "2016-09-30 00:00:00" & building_id == 1232 )')
train = train.query('not (meter == 3 & timestamp <= "2016-02-26 00:00:00" & building_id == 1227 )')
train = train.query('not (meter == 2 & timestamp <= "2016-02-15 00:00:00" & building_id == 1189 )')
train = train.query('not (meter == 2 & timestamp <= "2016-02-11 00:00:00" & building_id == 1176 )')
train = train.query('not (meter == 1 & timestamp >= "2016-09-01 00:00:00" & timestamp <= "2016-09-22 00:00:00" & building_id == 1226 )')
train = train.query('not (meter == 1 & timestamp >= "2016-09-01 00:00:00" & timestamp <= "2016-09-22 00:00:00" & building_id == 1225 )')
train = train.query('not (meter == 0 & timestamp <= "2016-08-04 00:00:00"& building_id == 1169 )')
train = train.query('not (meter == 0 & timestamp <= "2016-01-21 00:00:00"& building_id == 1153 )')
train = train.query('not (meter == 0 & timestamp <= "2016-09-08 00:00:00"& building_id == 1094 )')
train = train.query('not (meter == 0 & timestamp <= "2016-10-25 14:00:00"& building_id == 1073 )')
train = train.query('not (meter == 0 & timestamp >= "2016-07-13 00:00:00" & timestamp <= "2016-11-20 00:00:00" & building_id == 1241 )')
train = train.query('not (meter == 0 & timestamp >= "2016-01-03 00:00:00" & timestamp <= "2016-03-09 00:00:00" & building_id == 1234 )')
train = train.query('not (meter == 0 & timestamp >= "2016-01-03 00:00:00" & timestamp <= "2016-03-09 00:00:00" & building_id == 1233 )')
train = train.query('not (meter == 0 & timestamp >= "2016-09-14 00:00:00" & timestamp <= "2016-10-01 00:00:00" & building_id == 1232 )')
train = train.query('not (meter == 3 &  timestamp <= "2016-02-25 00:00:00" & building_id == 1227 )')
train = train.query('not (meter == 1 & timestamp >= "2016-08-25 00:00:00" & timestamp <= "2016-10-01 00:00:00" & building_id == 1226 )')
train = train.query('not (meter == 1 & timestamp >= "2016-08-25 00:00:00" & timestamp <= "2016-10-01 00:00:00" & building_id == 1225 )')
train = train.query('not (timestamp >= "2016-07-01 15:00:00" & timestamp <= "2016-07-05 04:00:00" & site_id == 9)')

In [ ]:
building_ids = [1250, 1331, 60, 1238, 1072, 783]
meters = [2, 2, 1, 2, 2, 2]
months = [12, 9, 5, 6, 8, 12]

for building_id, month, meter in zip(building_ids, months, meters):
    train = train[((train['building_id'] == building_id) & (train['month'] < month) & (train['meter'] <= meter)) == False]

In [ ]:
building_ids = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,30,32,34,36,38,40,42,44,45,47,49,50,51,52,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,106,112,190,259,455,681,761,777,799,802,895,604,48,29,1221,1169,1113,1094,1073,997,954,904,848,559,43,41,39,37,35,33,31,783,1331,1303,1290,1246,1246,1238,1160,1130,1072,1072,1003,954,954,236]
meters =       [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,2,1,3,1,2,1,1,2,1,1,2,1,1]
months =       [6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,7,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,11,11,2,4,6,5,9,5,11,9,5,11,5,8,10,7,7,8,10,6,8,2,4,3,5,5,5,5,5,5,5,11,9,6,2,2,2,5,10,9,7,3,2,8,8,3]

for building_id, month, meter in zip(building_ids, months, meters):
    train = train[((train['building_id'] == building_id) & (train['month'] < month) & (train['meter'] <= meter)) == False]

In [ ]:
train = train.query('not (meter == 0 & meter_reading == 0 )')

In [ ]:
train.info()

In [ ]:
train.to_pickle("train.pkl", compression='zip')

In [ ]:
test.to_pickle("test.pkl", compression='zip')
#test.info()

In [ ]:
assert(False)

In [8]:
train = pd.read_pickle("train.pkl", compression = 'zip')
test = pd.read_pickle("test.pkl", compression = 'zip')

In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19424763 entries, 103 to 20216099
Data columns (total 42 columns):
building_id                  int64
meter                        int64
timestamp                    datetime64[ns]
meter_reading                float64
site_id                      int64
primary_use                  int32
square_feet                  int64
year_built                   float64
floor_count                  float64
air_temperature              float64
cloud_coverage               float64
dew_temperature              float64
precip_depth_1_hr            float64
sea_level_pressure           float64
wind_direction               float64
wind_speed                   float64
hour                         int64
day                          int64
weekend                      int64
month                        int64
year                         int64
yearmonth                    int64
air_temperature_mean         float64
air_temperature_median       float64
air_temper

In [10]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41697600 entries, 0 to 41697599
Data columns (total 42 columns):
row_id                       int64
building_id                  int64
meter                        int64
timestamp                    datetime64[ns]
site_id                      int64
primary_use                  int32
square_feet                  int64
year_built                   float64
floor_count                  float64
air_temperature              float64
cloud_coverage               float64
dew_temperature              float64
precip_depth_1_hr            float64
sea_level_pressure           float64
wind_direction               float64
wind_speed                   float64
hour                         int64
day                          int64
weekend                      int64
month                        int64
year                         int64
yearmonth                    int64
air_temperature_mean         float64
air_temperature_median       float64
air_temperatur

## Outlier detection

In [ ]:
#import heapq
#for building_id in train['building_id'].unique():
#    for meter in train[train['building_id'] == building_id]['meter'].unique():
#        g = train[train['building_id'] == building_id][train['meter'] == meter]
#        largest = heapq.nlargest(5, g['meter_reading'])
#        if len(largest) > 2:
#            if largest[0] > largest[1] * 2:
#                #print(building_id, meter, largest[0], len(train))
#                train.loc[((train.building_id == building_id) & (train.meter_reading == largest[0]) & (train.meter == meter)), 'meter_reading'] = np.nan
#                #print(len(train))

In [11]:
def get_clf(site_id):
    if site_id in [0,1,2,3,4,15]:
        return lgb.LGBMRegressor(n_estimators=6000,learning_rate=0.2,feature_fraction=0.9,subsample=0.2,  subsample_freq=1,num_leaves=20, metric='rmse')
    elif site_id == 5:
        return lgb.LGBMRegressor(n_estimators=6000,learning_rate=0.17279828575424924,feature_fraction= 0.5014962543722731,max_depth=11,subsample=0.9679640078506427,num_leaves=10, metric='rmse')
    elif site_id == 6:
        return lgb.LGBMRegressor(n_estimators=6000,learning_rate=0.08080772525947343,feature_fraction= 0.622900028560716,max_depth=13,subsample=0.10067186032099355,num_leaves=34, metric='rmse')
    elif site_id == 7:
        return lgb.LGBMRegressor(n_estimators=6000,learning_rate=0.18299773578701567,feature_fraction= 0.6223879954919564,max_depth=10,subsample=0.9414223121432328,num_leaves=10, metric='rmse')
    elif site_id == 8:
        return lgb.LGBMRegressor(n_estimators=6000,learning_rate=0.09189872129725521,feature_fraction= 0.9753756863983394,max_depth=10,subsample=0.9832629640612986,num_leaves=11, metric='rmse')
    elif site_id == 9:
        return lgb.LGBMRegressor(n_estimators=6000,learning_rate=0.08255250790364155,feature_fraction= 0.5369661052821386,max_depth=16,subsample=0.9833827601905479,num_leaves=35, metric='rmse')
    elif site_id == 10:
        return lgb.LGBMRegressor(n_estimators=6000,learning_rate=0.2,feature_fraction= 0.5000000000000000,max_depth=8,subsample=1,num_leaves=21, metric='rmse')
    elif site_id == 11:
        return lgb.LGBMRegressor(n_estimators=6000,learning_rate=0.20000000000000000,feature_fraction= 0.5000000091689872,max_depth=10,subsample=0.6941231045450234,num_leaves=24, metric='rmse')
    elif site_id == 12:
        return lgb.LGBMRegressor(n_estimators=6000,learning_rate=0.17443329555575438,feature_fraction= 0.9734466427044206,max_depth=16,subsample=0.1137291435238815,num_leaves=10, metric='rmse')
    elif site_id == 13:
        return lgb.LGBMRegressor(n_estimators=6000,learning_rate=0.08338681267136139,feature_fraction= 0.5096109648903343,max_depth=15,subsample=0.9847653065775817,num_leaves=40, metric='rmse')
    elif site_id == 14:
        return lgb.LGBMRegressor(n_estimators=6000,learning_rate=0.08085570180603563,feature_fraction= 0.506472390324229,max_depth=10,subsample=0.702980869161743,num_leaves=21, metric='rmse')

## Train

In [21]:
num_folds = 3
#folds = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)
folds = KFold(n_splits = num_folds, shuffle = False, random_state = 42)
#train.dropna(subset=['meter_reading'], inplace=True)
t = []
tr_oof = []
a = 0
for site_id in train['site_id'].unique():
    a += 1
    print(site_id, meter)
    for meter in train[train['site_id'] == site_id]['meter'].unique():
        train_small = train[(train['site_id'] == site_id) & (train['meter'] == meter)]
        test_small = test[(test['site_id'] == site_id) & (test['meter'] == meter)]
        oof = np.zeros(len(train_small))
        predictions = np.zeros(len(test_small))
        target = np.log1p(train_small["meter_reading"])
        for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_small, train_small['month'])):
            print("---------", datetime.now(), site_id, meter, fold_)
            tr_x, tr_y = train_small.iloc[trn_idx][features], target.iloc[trn_idx]
            vl_x, vl_y = train_small.iloc[val_idx][features], target.iloc[val_idx]

            clf = get_clf(site_id)
            clf.fit(tr_x, tr_y,eval_set=[(vl_x, vl_y)],early_stopping_rounds=150,verbose=2000)
            oof[val_idx] = clf.predict(train_small.iloc[val_idx][features])
            predictions += np.expm1(clf.predict(test_small[features]))
        test_small['meter_reading_site_id'] = predictions / num_folds
        test_small['meter_reading_site_id'] = test_small['meter_reading_site_id'].clip(train_small["meter_reading"].min(),train_small["meter_reading"].max())
        train_small['meter_reading_p'] = oof
        t.append(test_small[['meter_reading_site_id']])
        tr_oof.append(train_small[['meter_reading_p']])
t = pd.concat(t)
t = t.sort_index()
oof = pd.concat(tr_oof)
oof = oof.sort_index()
#print(np.sqrt(mean_squared_error(target, oof['meter_reading_p'])))
try:del test['meter_reading_site_id']
except:None
test = test.merge(t, left_index=True, right_index=True, how='left')
target = np.log1p(train["meter_reading"])
print(np.sqrt(mean_squared_error(target, oof['meter_reading_p'])))
#0.8852894403996184

1 0
--------- 2020-02-02 11:31:33.248983 1 0 0
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[55]	valid_0's rmse: 0.304143
--------- 2020-02-02 11:31:41.975987 1 0 1
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[205]	valid_0's rmse: 0.458182
--------- 2020-02-02 11:31:57.723373 1 0 2
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[362]	valid_0's rmse: 0.632586
--------- 2020-02-02 11:32:22.587321 1 3 0
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[31]	valid_0's rmse: 1.46104
--------- 2020-02-02 11:32:25.546334 1 3 1
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[14]	valid_0's rmse: 1.71892
--------- 2020-02-02 11:32:28.289335 1 3 2
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[27]	v

[26]	valid_0's rmse: 0.500901
9 0
--------- 2020-02-02 11:49:28.060302 9 1 0
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[255]	valid_0's rmse: 1.17944
--------- 2020-02-02 11:50:04.015332 9 1 1
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[43]	valid_0's rmse: 0.770195
--------- 2020-02-02 11:50:20.223349 9 1 2
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[51]	valid_0's rmse: 1.16531
--------- 2020-02-02 11:50:40.860367 9 2 0
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[65]	valid_0's rmse: 1.3467
--------- 2020-02-02 11:50:58.706383 9 2 1
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[329]	valid_0's rmse: 1.17716
--------- 2020-02-02 11:51:38.954422 9 2 2
Training until validation scores don't improve for 150 rounds
Early stopping

[275]	valid_0's rmse: 1.80663
--------- 2020-02-02 12:11:14.026531 14 2 0
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[40]	valid_0's rmse: 1.25138
--------- 2020-02-02 12:11:20.052534 14 2 1
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[300]	valid_0's rmse: 1.11395
--------- 2020-02-02 12:11:35.426547 14 2 2
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[104]	valid_0's rmse: 1.06056
15 2
--------- 2020-02-02 12:11:47.342361 15 3 0
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[9]	valid_0's rmse: 0.575415
--------- 2020-02-02 12:11:48.551363 15 3 1
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[19]	valid_0's rmse: 0.729539
--------- 2020-02-02 12:11:49.891366 15 3 2
Training until validation scores don't improve for 150 rounds
Early 

In [22]:
num_folds = 4
#folds = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)
folds = KFold(n_splits = num_folds, shuffle = False, random_state = 42)
#train.dropna(subset=['meter_reading'], inplace=True)
t = []
tr_oof = []
a = 0
for building_id in train['building_id'].unique():
    a += 1
    print(a)
    for meter in train[train['building_id'] == building_id]['meter'].unique():
        train_small = train[(train['building_id'] == building_id) & (train['meter'] == meter)]
        test_small = test[(test['building_id'] == building_id) & (test['meter'] == meter)]
        oof = np.zeros(len(train_small))
        predictions = np.zeros(len(test_small))
        target = np.log1p(train_small["meter_reading"])
        for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_small, train_small['month'])):
            tr_x, tr_y = train_small.iloc[trn_idx][features], target.iloc[trn_idx]
            vl_x, vl_y = train_small.iloc[val_idx][features], target.iloc[val_idx]

            clf = lgb.LGBMRegressor(n_estimators=6000,
                                        learning_rate=0.2,
                                        feature_fraction=0.9,
                                        subsample=0.2,  
                                        subsample_freq=1,
                                        num_leaves=20,
                                        metric='rmse')
            clf.fit(tr_x, tr_y,eval_set=[(vl_x, vl_y)],early_stopping_rounds=150,verbose=2000)
            oof[val_idx] = clf.predict(train_small.iloc[val_idx][features])
            predictions += np.expm1(clf.predict(test_small[features]))
        test_small['meter_reading_building_id'] = predictions / num_folds
        test_small['meter_reading_building_id'] = test_small['meter_reading_building_id'].clip(train_small["meter_reading"].min(),train_small["meter_reading"].max())
        train_small['meter_reading_p'] = oof
        t.append(test_small[['meter_reading_building_id']])
        tr_oof.append(train_small[['meter_reading_p']])
t = pd.concat(t)
t = t.sort_index()
oof = pd.concat(tr_oof)
oof = oof.sort_index()
#print(np.sqrt(mean_squared_error(target, oof['meter_reading_p'])))
try:del test['meter_reading_building_id']
except:None
test = test.merge(t, left_index=True, right_index=True, how='left')
target = np.log1p(train["meter_reading"])
print(np.sqrt(mean_squared_error(target, oof['meter_reading_p'])))

1
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[18]	valid_0's rmse: 0.239692
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[12]	valid_0's rmse: 0.166305
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[6]	valid_0's rmse: 0.153771
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[41]	valid_0's rmse: 0.248251
2
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[40]	valid_0's rmse: 0.9931
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[8]	valid_0's rmse: 0.97952
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[20]	valid_0's rmse: 0.306796
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[21]	valid_0's 

Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[188]	valid_0's rmse: 0.0747347
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[12]	valid_0's rmse: 0.0546938
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[16]	valid_0's rmse: 0.0517961
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[11]	valid_0's rmse: 0.0652968
13
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[15]	valid_0's rmse: 0.16533
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[20]	valid_0's rmse: 0.127598
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[56]	valid_0's rmse: 0.162019
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[32]	va

Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[16]	valid_0's rmse: 0.186448
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[4]	valid_0's rmse: 0.174301
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[218]	valid_0's rmse: 0.221098
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[4]	valid_0's rmse: 0.279069
26
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[159]	valid_0's rmse: 0.152924
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[9]	valid_0's rmse: 0.117513
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[22]	valid_0's rmse: 0.148804
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[13]	valid_0

Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[57]	valid_0's rmse: 0.250137
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[10]	valid_0's rmse: 0.181476
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[8]	valid_0's rmse: 0.153839
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[145]	valid_0's rmse: 0.24031
40
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[1]	valid_0's rmse: 1.7559
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[49]	valid_0's rmse: 0.715833
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[6]	valid_0's rmse: 1.00994
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[1]	valid_0's rms

Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[25]	valid_0's rmse: 0.234639
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[31]	valid_0's rmse: 0.193494
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[39]	valid_0's rmse: 0.198132
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[11]	valid_0's rmse: 0.155477
54
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[8]	valid_0's rmse: 0.277101
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[39]	valid_0's rmse: 0.207696
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[26]	valid_0's rmse: 0.195715
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[41]	valid_0

...

[123]	valid_0's rmse: 0.259977
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[19]	valid_0's rmse: 0.144898
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[60]	valid_0's rmse: 0.151914
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[1]	valid_0's rmse: 0.252201
1393
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[130]	valid_0's rmse: 0.228582
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[12]	valid_0's rmse: 0.140968
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[59]	valid_0's rmse: 0.162524
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[1]	valid_0's rmse: 0.35278
1394
Training until validation scores don't improve for 150 rounds
Early stop

[29]	valid_0's rmse: 0.172336
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[10]	valid_0's rmse: 0.141132
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[15]	valid_0's rmse: 0.203322
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[60]	valid_0's rmse: 0.221015
1409
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[195]	valid_0's rmse: 0.0963075
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[16]	valid_0's rmse: 0.0659436
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[71]	valid_0's rmse: 0.109736
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[23]	valid_0's rmse: 0.153364
1410
Training until validation scores don't improve for 150 rounds
Early 

[15]	valid_0's rmse: 0.162026
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[25]	valid_0's rmse: 0.171749
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[18]	valid_0's rmse: 0.226756
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[13]	valid_0's rmse: 0.30258
1425
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[25]	valid_0's rmse: 0.203434
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[9]	valid_0's rmse: 0.172399
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[8]	valid_0's rmse: 0.120114
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[43]	valid_0's rmse: 0.178877
1426
Training until validation scores don't improve for 150 rounds
Early stoppi

[26]	valid_0's rmse: 0.295213
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[37]	valid_0's rmse: 0.423622
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[61]	valid_0's rmse: 0.539451
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[21]	valid_0's rmse: 0.346031
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[46]	valid_0's rmse: 0.18198
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[44]	valid_0's rmse: 0.246235
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[18]	valid_0's rmse: 0.161702
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[42]	valid_0's rmse: 0.233177
1437
Training until validation scores don't improve for 150 rounds
Early stopping,

[244]	valid_0's rmse: 0.0360551
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[154]	valid_0's rmse: 0.0300583
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[80]	valid_0's rmse: 0.0297729
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[27]	valid_0's rmse: 0.0373637
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[2]	valid_0's rmse: 0.157989
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[24]	valid_0's rmse: 0.0955644
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[29]	valid_0's rmse: 0.0997268
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[6]	valid_0's rmse: 0.131752
Training until validation scores don't improve for 150 rounds
Early stoppin

In [23]:
num_folds = 4
#folds = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)
folds = KFold(n_splits = num_folds, shuffle = False, random_state = 42)
#train.dropna(subset=['meter_reading'], inplace=True)
t = []
tr_oof = []
a = 0
for primary_use in train['primary_use'].unique():
    a += 1
    print(a)
    for meter in train[train['primary_use'] == primary_use]['meter'].unique():
        train_small = train[(train['primary_use'] == primary_use) & (train['meter'] == meter)]
        test_small = test[(test['primary_use'] == primary_use) & (test['meter'] == meter)]
        oof = np.zeros(len(train_small))
        predictions = np.zeros(len(test_small))
        target = np.log1p(train_small["meter_reading"])
        for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_small, train_small['month'])):
            tr_x, tr_y = train_small.iloc[trn_idx][features], target.iloc[trn_idx]
            vl_x, vl_y = train_small.iloc[val_idx][features], target.iloc[val_idx]

            clf = lgb.LGBMRegressor(n_estimators=6000,
                                        learning_rate=0.2,
                                        feature_fraction=0.9,
                                        subsample=0.2,  
                                        subsample_freq=1,
                                        num_leaves=20,
                                        metric='rmse')
            clf.fit(tr_x, tr_y,eval_set=[(vl_x, vl_y)],early_stopping_rounds=150,verbose=2000)
            oof[val_idx] = clf.predict(train_small.iloc[val_idx][features])
            predictions += np.expm1(clf.predict(test_small[features]))
        test_small['meter_reading_primary_use'] = predictions / num_folds
        test_small['meter_reading_primary_use'] = test_small['meter_reading_primary_use'].clip(train_small["meter_reading"].min(),train_small["meter_reading"].max())
        train_small['meter_reading_p'] = oof
        t.append(test_small[['meter_reading_primary_use']])
        tr_oof.append(train_small[['meter_reading_p']])
t = pd.concat(t)
t = t.sort_index()
oof = pd.concat(tr_oof)
oof = oof.sort_index()
#print(np.sqrt(mean_squared_error(target, oof['meter_reading_p'])))
try:del test['meter_reading_primary_use']
except:None
test = test.merge(t, left_index=True, right_index=True, how='left')
target = np.log1p(train["meter_reading"])
print(np.sqrt(mean_squared_error(target, oof['meter_reading_p'])))

1
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[147]	valid_0's rmse: 0.367448
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[1018]	valid_0's rmse: 0.301448
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[655]	valid_0's rmse: 0.31789
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[505]	valid_0's rmse: 0.413097
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[76]	valid_0's rmse: 1.20855
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[276]	valid_0's rmse: 1.30518
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[60]	valid_0's rmse: 1.19244
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[16]	valid

[278]	valid_0's rmse: 0.318254
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[887]	valid_0's rmse: 0.370869
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[53]	valid_0's rmse: 0.408406
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[236]	valid_0's rmse: 0.400256
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[71]	valid_0's rmse: 1.52169
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[209]	valid_0's rmse: 1.30991
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[430]	valid_0's rmse: 1.10504
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[16]	valid_0's rmse: 1.56081
Training until validation scores don't improve for 150 rounds
Early stopping, be

Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[6]	valid_0's rmse: 0.538124
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[156]	valid_0's rmse: 0.551875
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[228]	valid_0's rmse: 0.478125
11
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[103]	valid_0's rmse: 0.211852
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[202]	valid_0's rmse: 0.167098
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[220]	valid_0's rmse: 0.149988
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[175]	valid_0's rmse: 0.214036
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[67]	v

[7]	valid_0's rmse: 1.2035
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[23]	valid_0's rmse: 1.07021
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[74]	valid_0's rmse: 0.481542
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[27]	valid_0's rmse: 3.76404
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[14]	valid_0's rmse: 0.239549
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[24]	valid_0's rmse: 0.188701
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[1]	valid_0's rmse: 0.203624
16
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[13]	valid_0's rmse: 0.211999
Training until validation scores don't improve for 150 rounds
Early stopping, best i

In [25]:
sub = pd.read_csv(DATA_PATH + "sample_submission.csv")
#sub["meter_reading"] = (test['meter_reading_site_id'] * 0.6 + test['meter_reading_primary_use'] * 0.2 + test['meter_reading_building_id'] * 0.2)
#sub.to_csv("best_more_deleting.gz",  index = False, compression= 'gzip')

In [26]:
test["meter_reading"] = test['meter_reading_site_id'] * 0.6 + test['meter_reading_primary_use'] * 0.2 + test['meter_reading_building_id'] * 0.2

In [27]:
###Postprocess 
test['meter_reading'] = [meter_r * 0.85 if site not in [0,1,2,3,4,15] else meter_r for meter_r,site in zip(test['meter_reading'], test['site_id'])]

In [28]:
leak_df = pd.read_csv(LEAKDATA_PATH+'leak.csv')
leak_df['timestamp'] = pd.to_datetime(leak_df["timestamp"])
leak_df.dropna(0, inplace=True)
leak_df = leak_df[(leak_df.timestamp.dt.year > 2016) & (leak_df.timestamp.dt.year < 2019)]
leak_df.loc[leak_df.meter_reading < 0, 'meter_reading'] = 0 # remove large negative values
leak_df = leak_df[leak_df.building_id!=245]
leak_df['meter_reading_sc'] = leak_df['meter_reading'] 
del leak_df['meter_reading'] 
test = test.merge(leak_df, on=['building_id','meter','timestamp'], how='left')

In [ ]:
#test[['row_id', 'meter_reading_all','meter_reading_site_id','meter_reading_building_id','meter_reading_primary_use','meter_reading_sc']].to_pickle('test_pred09122019.pkl')

In [29]:
test['meter_reading'] = [a if (a >= 0 and site != 4) or (a < 0 and site != 4) else b for a,b,site in zip(test['meter_reading_sc'], test['meter_reading'], test['site_id'])]

In [30]:
sub["meter_reading"] = test['meter_reading']
sub.to_csv("../sub/lgb_3wise.csv",  index = False,float_format="%.4f")